In [ ]:
%%sql
postgresql:///animal_shelters

WITH ani AS (
SELECT 
    a.animaltype AS animaltype,
    a.birthdate AS birthdate,
    a.location AS location,
    (CASE WHEN a.animaltype = 'Dog' AND weight <= 10 THEN 'Small'
          WHEN a.animaltype = 'Dog' AND weight > 10 AND weight <= 30 THEN 'Medium'
          WHEN a.animaltype = 'Dog' AND weight > 30 THEN 'Large'
          WHEN a.animaltype = 'Cat' AND weight <= 5 THEN 'Small'
          WHEN a.animaltype = 'Cat' AND weight > 5 AND weight <= 7 THEN 'Medium'
          WHEN a.animaltype = 'Cat' AND weight > 7 THEN 'Large'
          WHEN a.animaltype = 'Bird' AND weight <= 0.7 THEN 'Small'
          WHEN a.animaltype = 'Bird' AND weight > 0.7 AND weight <= 1.1 THEN 'Medium'
          ELSE 'Large' 
     END
    ) AS size,
    (date '2020-12-31' - TO_DATE(birthdate, 'MM/DD/YYYY') )/365 AS age
FROM animals AS a
LEFT JOIN sponsored_animals AS sp
ON a.animalid = sp.sponsorid
WHERE sponsorid IS NULL)

SELECT  a.animaltype,
        a.size,
        (lc.costs + sc.costs + ac.costs) AS total,
        ROUND(sc.costs * 100.0 / (lc.costs + sc.costs + ac.costs),2) AS percentage
FROM ani AS a
LEFT JOIN size_costs AS sc
ON a.animaltype = sc.animaltype AND a.size = sc.size
LEFT JOIN location_costs AS lc
ON a.location = lc.location
LEFT JOIN age_costs AS ac
ON ac.age LIKE CAST(a.age AS varchar(5))
WHERE a.size IN ('Small', 'Medium','Large')
ORDER  BY animaltype,
        CASE when a.size = 'Small' then 1
            when a.size = 'Medium' then 2
            when a.size = 'Large' then 3
            ELSE NULL 
        END